In [ ]:
%load_ext autoreload
%autoreload 2

import boto3
import sagemaker
import pandas as pd
import os
import sys

sys.path.insert(0,'..')

import src.config as cf
import src.utils as ut

os.environ['HTTP_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['HTTPS_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['no_proxy'] = "169.254.169.254,127.0.0.1"

# Params

In [ ]:
RUN_ENV = "dev"
freq = 'W'
prediction_length = 16
cutoff_week_id = 201922

In [ ]:
config_file = "../conf/prod.yml" if RUN_ENV == "prod" else "../conf/dev.yml"
config = cf.ProgramConfiguration(config_file, "../conf/functional.yml")

sagemaker_session = sagemaker.Session()
role = config.get_global_role_arn()

bucket = config.get_train_bucket_input()
prefix = 'test_deepAR' # Your S3 test dir

s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

model_name = "APO-{}-DEEPAR".format(cutoff_week_id)

# Get DeepAR Image

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

# Run Training Job

In [ ]:
job_name = "TRAINING-{}".format(model_name)
output_path = "s3://{}/model".format(s3_output_path)

estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    base_job_name=job_name,
    output_path=output_path,
    train_use_spot_instances=True,
    train_max_run=3600,
    train_max_wait=3600
)

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "prediction_length": str(prediction_length),
    "context_length": "21",
    "num_cells": "199",
    "num_layers": "3",
    "likelihood": "negative-binomial",
    "epochs": "126",
    "mini_batch_size": "256",
    "learning_rate": "9.958020405900597e-05",
    "dropout_rate": "0.17555152252766837",
    "test_quantiles" : "[0.5]",
    "early_stopping_patience" : "10"
}

estimator.set_hyperparameters(**hyperparameters)

In [ ]:
inputs = {
    "train" : "s3://{}/cutoff_{}/val.json".format(s3_data_path, cutoff_week_id)
}

estimator.fit(inputs=inputs)

# Create Model from training job

In [ ]:
# Delete old model version if exists
try: 
    sagemaker_session.delete_model(model_name)
except:
    pass

training_job_name = estimator.latest_training_job.name

model = sagemaker_session.create_model_from_job(
    training_job_name,
    name=model_name,
    role=role,
)